In [2]:
import json
# s3 params
with open('s3_params.txt', 'r') as f:
    s3_params = json.loads(f.read())
s3_key = s3_params['access-key']
s3_secret = s3_params['secret-key']
s3_url = s3_params['endpoint_url']
s3_binance_cred_path = s3_params['cred_path']

In [29]:
from tensorflow.keras.applications import ResNet50

In [4]:
# Read schema from file
with open('schema.txt', 'r') as f:
    schema_str = f.read()

In [4]:
from s3fs.core import S3FileSystem
# s3fs client
s3 = S3FileSystem(
     anon=False,
     key=s3_key,
     secret=s3_secret,
     use_ssl=False,
     client_kwargs={'endpoint_url': s3_url})

In [8]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

config = {
    "spark.kubernetes.namespace": "spark",
    "spark.kubernetes.container.image": "localhost:32000/spark:3.1.2-hadoop-3.2.0-aws",
    "spark.executor.instances": "1",
    "spark.executor.memory": "1g",
    "spark.executor.cores": "1",
    "spark.driver.blockManager.port": "7777",
    "spark.driver.port": "2222",
    "spark.driver.host": "jupyter.spark.svc.cluster.local",
    "spark.driver.bindAddress": "0.0.0.0",
    "spark.hadoop.fs.s3a.endpoint": s3_url,
    "spark.hadoop.fs.s3a.access.key": s3_key,
    "spark.hadoop.fs.s3a.secret.key": s3_secret,
    "spark.hadoop.fs.s3a.connection.ssl.enabled": "false",
    "spark.hadoop.fs.s3a.path.style.access": "true",
    "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.com.amazonaws.services.s3.enableV4": "true",
    "spark.hadoop.fs.s3a.aws.credentials.provider": "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    "spark.hadoop.fs.s3a.committer.name": "directory",
    "spark.sql.sources.commitProtocolClass": "org.apache.spark.internal.io.cloud.PathOutputCommitProtocol",
    "spark.sql.parquet.output.committer.class": "org.apache.spark.internal.io.cloud.BindingParquetOutputCommitter"
}

def get_spark_session(app_name: str, conf: SparkConf):
    conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local")
    for key, value in config.items():
        conf.set(key, value)    
    return SparkSession.builder.appName(app_name).config(conf=conf).getOrCreate()

In [9]:
spark = get_spark_session("aws_localstack", conf)

21/10/12 08:53:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
try:
    df = spark.read.csv('s3a://data/test/BTCUSDT_MinuteBars.csv', header=True)
    df.printSchema()
    print(df.count())
except Exception as exp:
    print(exp)

spark.stop()

In [10]:
spark = get_spark_session("localstack", conf)

try:
    df = spark.read.parquet('s3a://data/test/test.parquet/')
    df.printSchema()
    print(df.count())
except Exception as exp:
    print(exp)

#spark.stop()

21/10/12 08:53:52 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


root
 |-- hoge: double (nullable = true)
 |-- foo: string (nullable = true)
 |-- bar: double (nullable = true)
 |-- boo: string (nullable = true)



2


In [11]:
from typing import Any, Dict, List
from s3fs.core import S3FileSystem
from pyarrow import Table, parquet as pq
from pandas import DataFrame, Series

def to_df(data: List[Dict[str, Any]]) -> DataFrame:                                                                                                         
    df = DataFrame()
    for item in data:
        indexes = []
        values = []
        for k, v in item.items():
            indexes.append(k)
            values.append(v)
        df = df.append(Series(values, index=indexes), ignore_index=True)
    return df

# s3fs client
fs = S3FileSystem(
     anon=False,
     key=s3_key,
     secret=s3_secret,
     use_ssl=False,
     client_kwargs={'endpoint_url': s3_url})

path_to_s3_object = "s3://data/test/test.parquet"

data = [
  {
    "hoge": 1,
    "foo": "blah",
  },
  {
    "boo": "test",
    "bar": 123,
  },
]
df = to_df(data)
pq.write_to_dataset(
    Table.from_pandas(df),
    path_to_s3_object,
    filesystem=fs,
    use_dictionary=True,
    #compression="snappy",
    version="2.0",
)

In [1]:
import argparse

from confluent_kafka import DeserializingConsumer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer
from confluent_kafka.serialization import StringDeserializer

In [ ]:
def main(args):
    
    topic = args.topic

    schema_registry_conf = {'url': args.schema_registry}

    schema_registry_client = SchemaRegistryClient(schema_registry_conf)

    avro_deserializer = AvroDeserializer(schema_registry_client, schema_str)
    
    string_deserializer = StringDeserializer('utf_8')

    consumer_conf = {'bootstrap.servers': args.bootstrap_servers,
                     'key.deserializer': string_deserializer,
                     'value.deserializer': avro_deserializer,
                     'group.id': args.group,
                     'auto.offset.reset': "earliest"}

    consumer = DeserializingConsumer(consumer_conf)
    consumer.subscribe([topic])

    while True:
        try:
            # SIGINT can't be handled when polling, limit timeout to 1 second.
            msg = consumer.poll(1.0)
            print(msg.value())
            
        except KeyboardInterrupt:
            break

    consumer.close()


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="AvroDeserializingProducer")
    parser.add_argument('-b', dest="bootstrap_servers",
                        default="kafka.kafka.svc.cluster.local:9092", help="Bootstrap servers")
    parser.add_argument('-r', dest="schema_registry",
                        default="http://schema-registry.kafka.svc.cluster.local:8085", help="Schema registry url")
    parser.add_argument('-t', dest="topic", default="test", help="Topic")
    parser.add_argument('-s', dest="symbol", default="BTCUSDT", help="Symbol")
    parser.add_argument('-g', dest="group", default="test", help="Consumer group")
    main(parser.parse_args())

In [ ]:
topic = "test"

schema_registry_conf = {"url": "http://schema-registry.kafka.svc.cluster.local:8085"}

schema_registry_client = SchemaRegistryClient(schema_registry_conf)

avro_deserializer = AvroDeserializer(schema_registry_client, schema_str)

string_deserializer = StringDeserializer('utf_8')

consumer_conf = {'bootstrap.servers': "kafka.kafka.svc.cluster.local:9092",
                 'key.deserializer': string_deserializer,
                 'value.deserializer': avro_deserializer,
                 'group.id': "test",
                 'auto.offset.reset': "earliest"}

consumer = DeserializingConsumer(consumer_conf)
consumer.subscribe([topic])

while True:
    try:
        # SIGINT can't be handled when polling, limit timeout to 1 second.
        msg = consumer.poll(1.0)
        print(msg.value())

    except KeyboardInterrupt:
        break

consumer.close()


from typing import Any, Dict, List
from s3fs.core import S3FileSystem
from pyarrow import Table, parquet as pq
from pandas import DataFrame, Series

def to_df(data: List[Dict[str, Any]]) -> DataFrame:                                                                                                         
    df = DataFrame()
    for item in data:
        indexes = []
        values = []
        for k, v in item.items():
            indexes.append(k)
            values.append(v)
        df = df.append(Series(values, index=indexes), ignore_index=True)
    return df

# s3fs client
fs = S3FileSystem(
     anon=False,
     key=s3_key,
     secret=s3_secret,
     use_ssl=False,
     client_kwargs={'endpoint_url': s3_url})

path_to_s3_object = "s3://data/test/test.parquet"

data = [
  {
    "hoge": 1,
    "foo": "blah",
  },
  {
    "boo": "test",
    "bar": 123,
  },
]
df = to_df(data)
pq.write_to_dataset(
    Table.from_pandas(df),
    path_to_s3_object,
    filesystem=fs,
    use_dictionary=True,
    #compression="snappy",
    version="2.0",
)

In [ ]:
topic = "test"

schema_registry_conf = {"url": "http://schema-registry.kafka.svc.cluster.local:8085"}

schema_registry_client = SchemaRegistryClient(schema_registry_conf)

avro_deserializer = AvroDeserializer(schema_registry_client, schema_str)

string_deserializer = StringDeserializer('utf_8')

consumer_conf = {'bootstrap.servers': "kafka.kafka.svc.cluster.local:9092",
                 'key.deserializer': string_deserializer,
                 'value.deserializer': avro_deserializer,
                 'group.id': "test",
                 #'auto.offset.reset': "smallest"
                }

consumer = DeserializingConsumer(consumer_conf)
consumer.subscribe([topic])

while True:
    try:
        # SIGINT can't be handled when polling, limit timeout to 1 second.
        msg = consumer.poll(1.0)
        if msg is None:
            print("none")
            continue
        
        if msg is not None:
            print(hash(str(msg.value())))

    except KeyboardInterrupt:
        break

consumer.close()

In [1]:
#!/usr/bin/env python

import json
from binance import ThreadedWebsocketManager
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import StringSerializer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer

# Read schema from file
with open('schema.txt', 'r') as f:
    schema_str = f.read()


topic = "avro-test2"
symbol = "BTCUSDT"

schema_registry_conf = {'url': "http://10.105.144.163:8085"}

schema_registry_client = SchemaRegistryClient(schema_registry_conf)

avro_serializer = AvroSerializer(schema_registry_client, schema_str)

producer_conf = {'bootstrap.servers': "10.102.117.121:9092",
                    'key.serializer': StringSerializer('utf_8'),
                    'value.serializer': avro_serializer}

producer = SerializingProducer(producer_conf)

twm = ThreadedWebsocketManager()

def handle_socket_message(msg):

    print(msg)
    
    producer.produce(topic=topic, key=symbol, value=msg)
    producer.flush()


i = 0

while i < 3:
    
    i = i+1
 
    twm.start()
        
    client = twm.start_kline_socket(callback=handle_socket_message, symbol=symbol)

    # twm.join()
    
    twm.stop_socket(client)
        


RuntimeError: threads can only be started once